In [ ]:
import pandas as pd
import numpy as np
import nltk
import hazm
from hazm import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.decomposition import FactorAnalysis
import pickle

In [ ]:
data = pd.read_csv('data/per.csv')
data.head()

In [ ]:
with open('data/stopwords.txt') as stopwords_file:
    stopwords = stopwords_file.readlines()
stopwords = [line.replace('\n', '') for line in stopwords]
stopwords

In [ ]:
nltk.download('stopwords')

In [ ]:
nltk_stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(nltk_stopwords)
len(stopwords)

In [ ]:
stemmer = hazm.Stemmer()

In [ ]:
stemmer.stem('کتاب‌ها')

In [ ]:
dataset = pd.DataFrame(columns=('title_body', 'category'))
for index, row in data.iterrows():
    title_body = row['Title'] + ' ' + row['Body']
    title_body_tokenized = word_tokenize(title_body)
    title_body_tokenized_filtered = [w for w in title_body_tokenized if not w in stopwords]
    title_body_tokenized_filtered_stemmed = [stemmer.stem(w) for w in title_body_tokenized_filtered]
    dataset.loc[index] = {
        'title_body': ' '.join(title_body_tokenized_filtered_stemmed),
        'category': row['Category2'].replace('\n', '')
    }

In [ ]:
dataset.head()

In [ ]:
vectorizer = TfidfVectorizer()
vectorizer.fit(dataset['title_body'])

In [ ]:
X = vectorizer.transform(dataset['title_body'])

In [ ]:
X

In [ ]:
np.random.seed(1232)

In [ ]:
sample_index = np.random.randint(X.shape[0], size=1000)

In [ ]:
X = X[sample_index, :]

In [ ]:
X = X[:, np.random.randint(X.shape[1], size=1000)]

In [ ]:
X.shape

In [ ]:
le = LabelEncoder()
y = le.fit_transform(dataset['category'])

In [ ]:
le.classes_

In [ ]:
y = y[sample_index]

In [ ]:
len(y)

In [ ]:
print(np.unique(dataset['category']))

In [ ]:
np.shape(X)

In [ ]:
np.shape(y)

In [ ]:
with open('x.txt', 'wb') as f:
    pickle.dump(X, f)

In [ ]:
with open('y.txt', 'wb') as f:
    pickle.dump(y, f)

In [ ]:
with open('x.txt', 'rb') as f:
    x = pickle.load(f)

In [ ]:
with open('y.txt', 'rb') as f:
    y = pickle.load(f)

In [ ]:
fa = FactorAnalysis(n_components=100)

In [ ]:
fa.fit(X.toarray())

In [ ]:
X = fa.transform(X.toarray())

In [ ]:
np.shape(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
svmc = svm.SVC()
svmc.fit(X_train, y_train)

In [ ]:
svmc.score(X_test, y_test)